In [2]:
sc

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-4>

In [3]:
f = sc.wholeTextFiles('s3://cpsc5330s21/data-input/textcorpora-SMALL/')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
import re
def filename(path):
  return re.match('s3://cpsc5330s21/data-input/textcorpora-SMALL/(.+)\.txt', path)[1]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
doc_id_and_all_its_terms = f.map(lambda t: (filename(t[0]), t[1].split()))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
def tokenPairs(docid, tokens):
  out = []
  for token in tokens:
    item = "".join(filter(lambda c: 97 <= ord(c) <= 122, token.lower()))
    if item != "":
        out.append((docid, item,))
  return out

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
doc_id_term_token = doc_id_and_all_its_terms.flatMap(lambda t: tokenPairs(t[0], t[1])) # [('austen-emma', 'emma'), ('austen-emma', 'by'), ('austen-emma', 'jane'),
doc_id_term_token.take(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('austen-emma', 'emma'), ('austen-emma', 'by'), ('austen-emma', 'jane')]

In [8]:
doc_id_term_token_counts = doc_id_term_token.map(lambda item: ((item[0] ,item[1]), 1,)).reduceByKey(lambda a,b: a+b) # [(('austen-emma', 'emma'), 751), (('austen-emma', 'by'), 567), (('austen-emma', 'austen'), 1),
doc_id_term_token_counts.take(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[(('austen-emma', 'emma'), 751), (('austen-emma', 'by'), 567), (('austen-emma', 'austen'), 1), (('austen-emma', 'volume'), 3), (('austen-emma', 'woodhouse'), 263)]

In [9]:
docid_term_count = doc_id_term_token.map(lambda item: (item[0], 1,)).reduceByKey(lambda a,b: a+b) # [('carroll-alice', 26382), ('austen-emma', 158128), ('bible-kjv', 790029), 
docid_term_count.take(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('carroll-alice', 26382), ('austen-emma', 158128), ('bible-kjv', 790029), ('blake-poems', 6817), ('burgess-busterbrown', 15864)]

In [10]:
term_idf = doc_id_term_token.distinct().map(lambda t: (t[1],1)).reduceByKey(lambda a,b: a+b).sortByKey(ascending=True)
term_idf.take(5)
# term_idf.take(5)
#  [('a', 10), ('aaron', 2), ('aaronites', 1), ('aarons', 2), ('aback', 1)]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('a', 10), ('aaron', 2), ('aaronites', 1), ('aarons', 2), ('aback', 1)]

In [11]:
doc_id_term_token_counts_temp = doc_id_term_token_counts.map(lambda t: (t[0][0],(t[0][1], t[1]),))
doc_id_term_token_counts_temp.take(5) # [('austen-emma', ('emma', 751)), ('austen-emma', ('by', 567)), ('austen-emma', ('austen', 1)), ('austen-emma', ('volume', 3)), ('austen-emma', ('woodhouse', 263))]

doc_id_term_token_counts_temp = doc_id_term_token_counts_temp.join(docid_term_count)
doc_id_term_token_counts_temp.take(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('carroll-alice', (('alices', 12), 26382)), ('carroll-alice', (('adventures', 6), 26382)), ('carroll-alice', (('in', 366), 26382)), ('carroll-alice', (('wonderland', 3), 26382)), ('carroll-alice', (('chapter', 12), 26382))]

In [12]:
doc_id_term_token_counts_temp2 = doc_id_term_token_counts_temp.map(lambda t: (t[1][0][0],(t[1][0][1], t[0], t[1][1])))
doc_id_term_token_counts_temp2.take(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('alices', (12, 'carroll-alice', 26382)), ('adventures', (6, 'carroll-alice', 26382)), ('in', (366, 'carroll-alice', 26382)), ('wonderland', (3, 'carroll-alice', 26382)), ('chapter', (12, 'carroll-alice', 26382))]

In [13]:
doc_id_term_token_counts_temp3 = doc_id_term_token_counts_temp2.join(term_idf)
doc_id_term_token_counts_temp3.take(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('down', ((101, 'carroll-alice', 26382), 9)), ('down', ((1125, 'bible-kjv', 790029), 9)), ('down', ((123, 'chesterton-brown', 71624), 9)), ('down', ((361, 'melville-moby_dick', 211802), 9)), ('down', ((89, 'milton-paradise', 79645), 9))]

In [14]:
all_joined = doc_id_term_token_counts_temp3.map(lambda t: (t[0],t[1][1],t[1][0][0],t[1][0][1],t[1][0][2],))
all_joined.take(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('down', 9, 101, 'carroll-alice', 26382), ('down', 9, 1125, 'bible-kjv', 790029), ('down', 9, 123, 'chesterton-brown', 71624), ('down', 9, 361, 'melville-moby_dick', 211802), ('down', 9, 89, 'milton-paradise', 79645)]

In [15]:
tfidf = all_joined.map(lambda t: ((t[3],t[0]), 1000000*((t[2]/t[4])/t[1])))
tfidf.take(15)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[(('carroll-alice', 'down'), 425.3742029498227), (('bible-kjv', 'down'), 158.2220399504322), (('chesterton-brown', 'down'), 190.81127368852154), (('melville-moby_dick', 'down'), 189.3802282844879), (('milton-paradise', 'down'), 124.16208034263154), (('austen-emma', 'down'), 48.48392863165706), (('blake-poems', 'down'), 325.9824295470474), (('burgess-busterbrown', 'down'), 301.1710651650137), (('whitman-leaves', 'down'), 100.69802036846322), (('carroll-alice', 'rabbithole'), 113.71389583807141), (('carroll-alice', 'very'), 606.4741111363809), (('bible-kjv', 'very'), 36.14494601534318), (('chesterton-brown', 'very'), 288.54387728508135), (('melville-moby_dick', 'very'), 167.87167050148514), (('shakespeare-caesar', 'very'), 65.40115433037393)]

In [16]:
from pprint import pprint
def relavence(str_to_check):
    split_string = str_to_check.split()
    tokens = list(filter(lambda t: t != "", ["".join(filter(lambda c: 97 <= ord(c) <= 122, x.lower())) for x in split_string]))
    matched_tfidf = tfidf.filter(lambda t: t[0][1] in (tokens)).map(lambda t: (t[0][0], t[1])).reduceByKey(lambda a,b: int((a+b)/(len(split_string)))).sortBy(lambda t: -t[1]).take(5)
    
    return matched_tfidf

pprint(relavence("dead KING garden!"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('bible-kjv', 51),
 ('carroll-alice', 41),
 ('shakespeare-caesar', 31),
 ('whitman-leaves', 30),
 ('chesterton-brown', 26)]

In [17]:
from pprint import pprint
def relavence_like_example(str_to_check):
    split_string = str_to_check.split()
    tokens = list(filter(lambda t: t != "", ["".join(filter(lambda c: 97 <= ord(c) <= 122, x.lower())) for x in split_string]))
    matched_tfidf = tfidf.filter(lambda t: t[0][1] in (tokens)).map(lambda t: (t[0][0], t[1])).reduceByKey(lambda a,b: int((a+b))).sortBy(lambda t: -t[1]).take(5)
    
    return matched_tfidf

pprint(relavence_like_example("dead KING garden!"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('bible-kjv', 371),
 ('carroll-alice', 342),
 ('blake-poems', 173),
 ('chesterton-brown', 160),
 ('milton-paradise', 112)]